<a href="https://colab.research.google.com/github/quastarK/Tensorflow_colab/blob/main/Variable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 변수 (Variable)


텐서플로 옵티마이저는 경사 하강법과 같은 알고리즘에 따라 변수값을 효율적으로 업데이트하는 전문적인 연산이 있습니다.

tf.keras.optimizers.Optimizer을 보면 옵티마이저 사용방법이 설명되어 있습니다.

또한, read_value를 사용하여 현재 변수값을 명시적으로 읽어올 수 있습니다.

In [1]:
import tensorflow as tf

## 변수 생성

변수를 생성하려면 단순하게 초기값을 설정하면 됩니다.

In [2]:
my_variable = tf.Variable(tf.ones([1, 2, 3]))

In [3]:
print(my_variable)

<tf.Variable 'Variable:0' shape=(1, 2, 3) dtype=float32, numpy=
array([[[1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)>


tf.device 범위 (scope)가 유효하다면, 변수는 해당 장치에 위치합니다.

그렇지 않다면, 변수는 dtype에 호환되는 "가장 빠른" 장치에 위치합니다.
(GPU가 가용하다면 대부분의 변수들이 자동적으로 GPU에 위치한다는 의미 입니다.)


다음의 코드는 v라는 변수를 만들고 두 번째 GPU에 위치시킵니다.

In [14]:
with tf.device("/device:GPU:1"):      # colab의 경우 GPU가 하나라 1 지정해도 0에 지정됨
  v = tf.Variable(tf.zeros([10,10]))


In [16]:
print(v.device)
print(v)

/job:localhost/replica:0/task:0/device:GPU:0
<tf.Variable 'Variable:0' shape=(10, 10) dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>


### 변수의 사용

tf.Variable의 값을 사용하려면 단순히 tf.Tensor로 취급하면 된다.

같은 변수에 할당하려면 assign, assign_add 메소드와 tf.Variable 클래스에 있는 friends를 사용해야한다.

In [20]:
v = tf.Variable(0.0)
w = v+1

In [21]:
print(w)
print(v)

tf.Tensor(1.0, shape=(), dtype=float32)
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>


In [30]:
v = tf.Variable(0.0)
v.assign_add(1)
v.read_value() # 1.0

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

## 변수 추적

텐서플로에서 하나의 변수는 하나의 파이썬 객체입니다.
층, 모델, 옵티마이저, 그리고 다른 관련된 도구들을 작성할때 모델안에 있는 모든 변수의 리스트가 필요할 수 있습니다.

자신의 파이썬 코드에서 필요에 따라 변수들을 추적하는 동안 변수를 소유하는 자신의 클래스에 기본 클래스로 tf.Module 사용을 권장합니다.

tf.Module의 인스턴스는 variable와 다른 모듈들을 탐색할 수 있는 모델에서 접근할 수 있는 모든 (훈련가능한) 변수를 리턴하는 trainable_variables 메서드가 포함되어 있습니다.

층을 구현하고 있다면 tf.keras.Layer가 기본 클래스로 더 좋을 수 있습니다.
인터페이스를 구현하는 것은 층이 케라스에 완전하게 통합되기 때문에 model.fit과 잘 통합된 다른 API들을 사용할 수 있게 됩니다.
tf.keras.Layer의 변수 추적은 tf.Module의 변수 추적과 동일합니다.




In [40]:
class MyModuleOne(tf.Module):
  def __init__(self):
    self.v0 = tf.Variable(1.0)
    self.vs = [tf.Variable(x) for x in range(10)]


class MyOtherModule(tf.Module):
  def __init__(self):
    self.m = MyModuleOne()
    self.v = tf.Variable(10.0)


m = MyOtherModule()
print(len(m.variables))    # 12; 11은 m (self.v0 : 1, self.vs :10), m에서 다른것은 v에서 (self.v)
print(m.variables)
len(m.trainable_variables)

12
(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=10.0>, <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>, <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0>, <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=1>, <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=2>, <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=3>, <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=4>, <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=5>, <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=6>, <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=7>, <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=8>, <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=9>)


12